# Using PINNs for the Schrödinger Equation

### Time Dependent Schrödinger Equation



The workflow for surrogate modeling using PINNs is demonstrated with a quantum harmonic oscillator as a motivating example. A PINN is constructed for the solution of the time-dependent Schrödinger equation
\begin{equation}
i \frac{\partial \psi(\mathbf{r}, t)}{\partial t}-\hat{H} \psi(\mathbf{r}, t)=0
\end{equation} 
for a quantum harmonic oscillator consisting of the superposition of two eigenstates $ m,n $. In the one-dimensional case, the Hamiltonian is
\begin{equation}
\hat{H_x}=-\frac{1}{2}\frac{\partial^{2}}{\partial x^{2}}+\frac{\omega^{2}}{2}x^{2}
\end{equation}
The analytical solution $\psi(x,t) \in \mathbb{C}$ is
\begin{equation*}
\phi_0(x) = \sqrt[4]{\frac{\omega}{\pi}}\exp\left(-\frac{\omega x^2}{2}\right)
\end{equation*}
\begin{equation*}
\phi_1(x) = \phi_0(x) \sqrt{2\omega}x
\end{equation*}
For higher states,
\begin{equation*}
\phi_n(x) = \phi_0(x) \frac{1}{\sqrt{2^n n!}}Her_n(\sqrt{\omega}x)\exp\left(-i E_n t\right)
\end{equation*}
where $ Her_n(\sqrt{\omega}x) $ is the $ n $th Hermite polynomial,
and phase $ \exp\left(-i E_n t\right) $ where $ E_n = (n+\frac{1}{2})\omega $

The superposition is defined as
\begin{equation}
\psi_{m,n}(x,t) = \frac{1}{\sqrt{2}}\left( e^{\left(-i E_m t\right)}\phi_m(x) + e^{\left(-i E_n t\right)}\phi_n(x)\right)
\end{equation}

Since the neural network is constrained to $ \mathbb{R} $, the complex valued solution can be represented as 
\begin{equation}
\psi(x,t) = u + iv    
\end{equation} where $ u = \operatorname{Re}(\psi) $ and $ v=\operatorname{Im}(\psi) $
The time-dependent Schrödinger equation can be written as
\begin{equation}
    \left(-\frac{\partial v}{\partial t}+\frac{1}{2}\frac{\partial^{2} u}{\partial x^{2}}-\frac{\omega^{2}}{2}x^{2}\right) + i \left(\frac{\partial u}{\partial t}+\frac{1}{2}\frac{\partial^{2} v}{\partial x^{2}}-\frac{\omega^{2}}{2}x^{2}\right) = 0
\end{equation}

This can be generalised to three dimensions with 
\begin{equation*}
    \phi_n = \phi_n(x)\phi_n(y)\phi_n(z)
\end{equation*}
\begin{equation*}
    \hat{H} =\hat{H_x}+\hat{H_y}+\hat{H_z}
\end{equation*}


### Training
Training data is generated on a high resolution grid from the analytical solution described above. 
The surrogate model $ \psi_{net}: \mathbb{R}^{3+1}\mapsto \mathbb{R}^{2} $ is constructed, with inputs $ (\mathbf{r},t) $ and outputs $ (u,v) $.
The loss function $ L $ is given by 
\begin{equation}
L=L_{f}+L_{BC}+L_{IC}
\end{equation}

\begin{equation*}
L_{BC}=\frac{1}{N_{BC}} \sum_{i=1}^{N_{BC}}\left(\left|u\left(\mathbf{r}_{BC}^{i}, t_{BC}^{i}\right)-u^{i}\right|^{2}+\left|u\left(\mathbf{r}_{BC}^{i}, t_{BC}^{i}\right)-v^{i}\right|^{2}\right)
\end{equation*}
\begin{equation*}
L_{IC}=\frac{1}{N_{IC}} \sum_{i=1}^{N_{IC}}\left(\left|u\left(\mathbf{r}_{IC}^{i}, t_{IC}^{i}\right)-u^{i}\right|^{2}+\left|v\left(\mathbf{r}_{IC}^{i}, t_{IC}^{i}\right)-v^{i}\right|^{2}\right)
\end{equation*}
\begin{equation*}
L_{f}=\frac{1}{N_{f}} \sum_{i=1}^{N_{f}}\left|f_{net}\left(\mathbf{r}_{f}^{i}, t_{f}^{i}\right)\right|^{2}
\end{equation*}
At each training step, the loss function is calculated on $ N_f $ collocation points, sampled randomly from the grid.

### Training Considerations
#### Architecture
The architecture and associated hyperparameters have significant impact on the performance of the PINN. The learning model architecture can be customized depending on the nature of the domain. For example, CNNs, RNNs and GNNs can be used for spatial, temporal and interacting problems respectively. For the one-dimensional quantum harmonic oscillator workflow, we use a FCN with 7 layers of 10 neurons each. 
The choice of activation functions is constrained by the fact that they have to be $ (n+1) $-differentiable for $ n $-order PDEs. For the one-dimensional quantum harmonic oscillator problem we use $ \tanh $ activation functions because they are 3-differentiable.
Optimizer selection is important for convergence of the learner and to avoid solutions at local minima. It has been shown that a combination of Adam at early training and L-BFGS . We use the Adam optimizer only given the relatively simple nature of this problem.

#### Collocation Points
The accuracy of the NN increases with increase in density of collocation points. However, the computational resources required also increases exponentially with the increase in density of points. This can lead to training bottlenecks, especially for high dimensional systems. A trade-off has to be made between the desired accuracy and number of collocation points because training the NN on a large number of points may lead to overfitting and adversely affect generalisability. The distribution of points can also be customised according to the problem domain. Density could be increased around areas of sharp discontinuities to capture more information about the domain. For the one-dimensional quantum harmonic oscillator, we use 1000 points per unit length of the domain interior with random sampling.

![Collocation Error](fig/pinn_colloc_performance.png)

![Collocation Time](fig/pinn_colloc_time.png)


### Sample questions:


1.  Derive the analytical solution for QHO.

2.  1.  Write a simple numerical solver (eg Runge-Kutta) to simulate the
        time evolution of this QHO system.

    2.  Implement a simple neural network as a surrogate model for this
        system. How would you represent a complex valued wavefunction in
        a neural network? (started code for neural network will be
        provided)

    3.  Now add the Physics-Informed regularization term to the simple
        neural network implemented in part (b).

    4.  Compare the results from approaches (a),(b) and (c) with the
        analytical solution.

3.  TISE is an eigenvalue problem. How would you incorporate further
    constraints into PINNs to get solutions for specific eigenstates?

4.  What are the advantages and disadvantages of ML models like PINNs
    over traditional numerical solvers?

### Sample simulation:


This figure shows results inferred with a PINN for the 1D TD QHO system:

![Real and predicted simulation for 1D
QHO](fig/pinn_0_1_results.png)

![Density results for 1D QHO](fig/pinn_density_1d.png)


### Extensions:


-   Content about Bayesian statistics can be incorporated by using a
    Gaussian Process based surrogate model for the Schrödinger equation.
    Example: [
    https://karan.sh/GPNN_schrodingers_equation/](https://karan.sh/GPNN_schrodingers_equation/)

-   Because of the relevance of differential equations to different
    physical systems, this module can be extended to multiple courses
    like Classical Mechanics and Electrostatics/dynamics.

